En una determinada transmissió de dades entre dos terminals col·locats a una distància de 100 metres ens indiquen les següents característiques: (i) velocitat de transmissió 100Mbps, (ii) mida de la trama mitjana de 8kBytes, (iii) cablejat UTP CAT 5 (vprop = 2E8 m/s), (iv) mida trama ACK de 256 Bytes i (v) temps de processat en cada terminal de 5 microsegons. Identifica, per aquestes condicions, l’eficiència per RQ-Inactiva i per RQ continua, amb una finestra igual a 16. El temps de ACK i el de processat, són menyspreables? En tots dos casos tenim una probabilitat de error igual al 1%, Quin protocol serà el més eficient? (3 punts)

In [1]:
import math as m
import numpy as np

In [2]:
#dades
Vtxt = 100E6
Vprop = 2E8
Ldades = 8*1000*8
Lack = 256*8
Pe = 0.1
k = 16
Tproc = 5E-6
d = 100

In [3]:
Ttx = Ldades/Vtxt
Tack = Lack/Vtxt
Tprop = d/Vprop
Ttotal = Ttx + Tack + 2*(Tprop + Tproc)
Taprox = Ttx + 2*Tprop
a_real = (Tack/2 + Tprop + Tproc)/Ttx
a_ideal = Tprop/Ttx

In [4]:
# RQ inactiva

U_real = (1-Pe)/(1+2*a_real)
U_ideal = (1-Pe)/(1+2*a_ideal)

print("El valor de l'eficiencia real es de %.2f en tant per cent mentre \
      que el valor de l'eficiencia ideal es de %.2f " %(U_real*100, U_ideal*100))

El valor de l'eficiencia real es de 85.78 en tant per cent mentre       que el valor de l'eficiencia ideal es de 89.86 


In [5]:
# RQ continua
print(1+2*a_real)

1.0491875


__Exercici 5__. Dos routers estan separats una distància de 1000m i units per un cable de coure (vprop = 2E8m/s). Enviem paquets de dades de 1000 bits. Calcula la velocitat de transmissió per tenir una eficiència mínima del 50% fent servir RQ inactiva. Fes el mateix considerant una probabilitat d’error del 5%

In [6]:
import sympy as sp

In [7]:
sp.var('v_tx')
L = 1000 # longitud trama, bits
U = 0.5 # Eficiencia
distancia = 1000
v_prop = 2E8 # velocitat de propagació, m/s
t_prop = distancia/v_prop
t_tx = L /v_tx
f = t_tx/(t_tx + 2* t_prop) - 0.5
f

-0.5 + 1000/(v_tx*(1.0e-5 + 1000/v_tx))

In [8]:
v_txt = sp.solve(f, v_tx)
v_txt[0]

100000000.000000

100 Mb/s es la velocitat requerida

In [9]:
a = t_prop/t_tx
a

5.0e-9*v_tx

In [10]:
Perr = 0.05
f = (1-Perr)/(1+2*a) -0.5
f

-0.5 + 0.95/(1.0e-8*v_tx + 1)

In [11]:
v_txt2 = sp.solve(f, v_tx)
v_txt2

[90000000.0000000]

__Exercici 8__. Determina la eficiència de un enllaç punt a punt, wireless si tenim una finestra lliscant de valor 7. La distància entre equips és de 100 Km, la velocitat de transmissió és de 1Mb/s i la mida mitjana de paquets és de 1000 bytes. La probabilitat d’error és igual a 0.05. Quina estratègia és millor: Repetició selectiva o retrocedir N?

In [12]:
distancia = 100E3 # distancia en metres
c_tx = 1E6 # tassa de transferència b/s
L = 1000 * 8 # longitud trama en bits
c_prop = 3E8 # velocitat ona EM en aire aprox
Perr = 0.05
sp.var("U")
sp.var("a")
sp.var("K")

f1 = 1-Perr
f2 = (K * (1-Perr))/(1 + 2*a)
f1, f2

(0.95, 0.95*K/(2*a + 1))

In [13]:
t_prop = distancia/c_prop
t_txt = L/c_tx
a_num = t_prop/t_txt

K_num = 7
if K_num >= (1+2*a_num):
    print("U = ", f1)
    print("1+2a = ", 1+2*a_num)
else:
    print(f2.subs(K, 7).subs(a, a_num).evalf())

U =  0.95
1+2a =  1.0833333333333333


el valor de a es independent de si es repetició selectiva o retransmissió n

Per tots dos casos K > 1+2a

Per tant:

In [14]:
U_rN = (1-Perr)/(1+2*a_num *Perr)
print(U_rN)

0.946058091286307


__Exercici 6__. Dos ordinadors estan separats una distància de 100m i units per un cable de coure
UTP cat 6. Si implementem un protocol RQ inactiva, determina la mida necessària de les
trames perquè l’eficiència sigui del 70%. Fes el mateix considerant una probabilitat d’error del
1%.

In [17]:
distancia = 100 # metres
Eficiencia = 0.7 
Error = 0.01
C = 2E8 # velocitat de propagació en cable de coure metres/segon
sp.var("t_prop")
sp.var("V_tx")
sp.var("L_ACK")
sp.var("L_DATA")
sp.var("t_proc")
# calcul dels temps 
t_tx = L_DATA/V_tx
t_ack = L_ACK/V_tx
t_prop = distancia/C

U = t_tx/(t_tx + 2*t_prop + 2*t_proc + t_ack)
U

L_DATA/(V_tx*(L_ACK/V_tx + L_DATA/V_tx + 2*t_proc + 1.0e-6))

In [22]:
sp.init_printing()
f = U - Eficiencia
mida = sp.solve(f, L_DATA)
mida_trama = mida[0]

mida_trama_ideal = mida_trama.subs(L_ACK, 0).subs(t_proc, 0)
mida_trama, mida_trama_ideal

In [31]:
# juguem amb varies velocitats per veure com evoluciona la mida de la trama

v = np.array([1E4, 1E5, 1E6, 1E7, 1E8])
mida_numerica = []
for element in v:
    mida_numerica.append(mida_trama_ideal.subs(V_tx, element))
print("valors de la longitud de la trama: ")
print(mida_numerica)
print([np.ceil(e) for e in mida_numerica])


valors de la longitud de la trama: 
[0.0233333333333333, 0.233333333333333, 2.33333333333333, 23.3333333333333, 233.333333333333]
[1, 1, 3, 24, 234]


Què ens està dient aquest resultat??

Per velocitats de transmissió petites, i es clar, distàncies relativament curtes l'eficiència de RQ-Inactiva és MOLT ALTA!! 

Ara bé, quan la velocitat i/o la distància augmenten, l'eficiència d'aquest algorisme comença a baixar i calen altres per elevar el seu valor 

In [18]:
# considerant l'error

U_real = U*(1-Error)
U_real

0.99*L_DATA/(V_tx*(L_ACK/V_tx + L_DATA/V_tx + 2*t_proc + 1.0e-6))

__Exercici 7__. Determina la mida que ha de tenir una finestra i el nombre de bits necessaris per
codificar els números de seqüència si volem tenir una eficiència igual a 1. Considera que la
distància mitjana entre equips és de 100 metres. La connexió es fa mitjançant coure i la mida
mitjana de les trames és de 8000 Bytes. La velocitat de transmissió és igual a 1Mb/s. Si la
probabilitat d’error és 0, quina estratègia és millor: Repetició selectiva o retrocedir N?

In [27]:
eficiencia = 1
distancia = 100 # metres
v_prop = 2E8 # metres per segon
L_DATA = 8000 * 8 # mida de la trama en bits
v_tx = 1E6 # 1 Mb/s

Si la probabilitat d’error és 0, quina estratègia és millor: Repetició selectiva o retrocedir N?

Si la probabilitat d'error és 0, seran iguals, ja que repetició selectiva o retransmissió N (retrocedir N, o go_back_N) són algorismes que s'apliquen quan tenim una certa probabilitat d'error.

In [30]:
t_tx = L_DATA/v_tx
t_prop = distancia/v_prop
# càlcul de a:
a = t_prop/t_tx
# mida de la finestra mínima:

W = 1+2*a
print("mida de la finestra = ", np.ceil(W), "bits")


mida de la finestra =  2.0 bits


In [ ]:
identificadors = [0, 1, 2, 3]

## Calcul de la Fragmentació de datagrames de l'exercici del PWP

El camp de desplaçament de fragment indica la posició de les DADES al datagrama inicial, per tant, només considerem el desplaçament de les dades!!!

Si el datagrama inicial es de 1440B i la capçalera és de 20B aleshores tenim:

1440 - 20 = 1420B de dades

Si la mida màxima del datagrama és de 576B tenim que en aquest cas les dades tindran una mida de 576 - 20 = 556B. Atenció, 556B no es múltiple de 8. El valor més proper múltiple de 8 és 552B que dona 552/8 = 69 Words de 64 bits. La mida del datagrama serà de 552B de dades + 20B de capçalera = 572B

In [1]:
552 + 552 + 316

1420

__Exercici 1__. Genera a partir de l’adreça de xarxa 192.168.0.0/24 5 subxarxes. Una que pugui
donar servei a 100 equips. Una altra que doni servei a 50 i les altres, com a mínim a 10 equips.
Feu l’exercici també amb Packet tracer.

Partim de la IP 192.168.0.0/24. A partir d'aqui primer calculem la subxarxa per donar servei als 100 equips...  
Agafem els darrers 8 bits de la IP proporcionada per crear les subxarxes i veure els equips associats.

* 192.168.0. "0 0000000"/25 => 192.168.0.0/25 => Aquesta és la que agafo per donar el servei als primers 100 equips.
* 192.168.0. "1 0000000"/25 => 192.168.0.128/25 => Aquesta la faré servir per generar les altres subxarxes

En tots dos casos tenim $2^7-2 $ usuaris (126) 

A partir de la IP 192.168.0.128/25 genero ara la subxarxa que donarà servei a 50 equips. Només necessitem 1, per tant amb l'altra continuem amb el subnetting

* 192.168.0. "10 000000"/26 => 192.168.0.128/26 => Aquesta és la que agafo per donar servei als 50 equips 
* 192.168.0. "11 000000"/26 => 192.168.0.192/26

Ara, el nombre d'usuaris serà $2^6 - 2 = 62 $

Finalment, per la resta, partim de la IP 192.168.0.192/26 i generarem les subxarxes necessàries per tenir les 4 subxarxes restants que  ens donaran servei cada una a uns 10 equips => 4 bits d'usuaris

* 192.168.0. "1100 0000"/28 
* 192.168.0. "1101 0000"/28
* 192.168.0. "1110 0000"/28
* 192.168.0. "1111 0000"/28

__Exercici 2__. Genera deu subxarxes a partir de l’adreça 161.116.95.0/24.

Per generar 10 subxarxes necessito 4 bits 
* 161.116.95. "0000 0000"/28
* 161.116.90. "0001 0000"/28
* 161.116.90. "0010 0000"/28
* 161.116.90. "0011 0000"/28
* 161.116.90. "0100 0000"/28
* 161.116.90. "0101 0000"/28
* 161.116.90. "0110 0000"/28
* 161.116.90. "0111 0000"/28
* 161.116.90. "1000 0000"/28
* 161.116.90. "1001 0000"/28

Usuaris per a cada subxarxa:
* 161.116.90. "xxxx 0001"/28
* 161.116.90. "xxxx 0010"/28
* 161.116.90. "xxxx 0011"/28
* 161.116.90. "xxxx 0100"/28
* 161.116.90. "xxxx 0101"/28
* 161.116.90. "xxxx 0110"/28
* 161.116.90. "xxxx 0111"/28
* 161.116.90. "xxxx 1000"/28
* 161.116.90. "xxxx 1001"/28
* 161.116.90. "xxxx 1010"/28
* 161.116.90. "xxxx 1011"/28
* 161.116.90. "xxxx 1100"/28
* 161.116.90. "xxxx 1101"/28
* 161.116.90. "xxxx 1110"/28
